# Mini-projet Probabilités — Test du GCL **RANDU**


## Introduction

Ce notebook teste le générateur congruentiel linéaire (GCL) **RANDU** avec les paramètres suivants :
- $x_{n+1} \equiv (65539 \times x_n) \mod 2^{31}$
- $x_0 = 12345$

Nous allons tester ce générateur avec trois tailles d'échantillons :
- 30 000 données
- 90 000 données
- 300 000 données

Les tests appliqués sont :
1. **Test du khi-deux** (seuil de confiance à 1%)
2. **Test spectral 2D** (sur des doublets)
3. **Test spectral 3D** (sur des triplets)

## 1. Importation des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Implémentation du générateur RANDU

In [ ]:
def randu_generator(seed, n):
    """
    Générateur congruentiel linéaire RANDU
    x_{n+1} = (65539 * x_n) mod 2^31
    
    Parameters:
    -----------
    seed : int
        Graine initiale (x_0)
    n : int
        Nombre de valeurs à générer
    
    Returns:
    --------
    numpy.ndarray : Tableau de n nombres dans [0, 1]
    """
    a = 65539
    m = 2**31
    current = seed
    result = []
    
    for _ in range(n):
        current = (a * current) % m
        result.append(current / m)
    
    return np.array(result)

# Test du générateur
print("Premiers 10 nombres générés:")
test_data = randu_generator(12345, 10)
for i, val in enumerate(test_data, 1):
    print(f"x_{i} = {val:.6f}")

## 3. Test du Khi-deux (χ²)

Le test du khi-deux vérifie si la distribution des nombres générés suit une distribution uniforme.

Pour un seuil de confiance de 1% (α = 0.01), on rejette l'hypothèse nulle si la statistique χ² est supérieure à la valeur critique.

In [ ]:
def chi_square_test(data, num_bins=100, alpha=0.01):
    """
    Test du khi-deux pour vérifier l'uniformité
    
    Parameters:
    -----------
    data : array-like
        Données à tester (entre 0 et 1)
    num_bins : int
        Nombre de classes
    alpha : float
        Seuil de confiance (0.01 pour 1%)
    
    Returns:
    --------
    dict : Résultats du test
    """
    n = len(data)
    
    # Histogramme observé
    observed, _ = np.histogram(data, bins=num_bins, range=(0, 1))
    
    # Fréquences attendues (distribution uniforme)
    expected = np.full(num_bins, n / num_bins)
    
    # Statistique du khi-deux
    chi2_stat = np.sum((observed - expected)**2 / expected)
    
    # Degrés de liberté
    df = num_bins - 1
    
    # Valeur critique
    critical_value = stats.chi2.ppf(1 - alpha, df)
    
    # P-value
    p_value = 1 - stats.chi2.cdf(chi2_stat, df)
    
    # Décision
    reject_null = chi2_stat > critical_value
    
    return {
        'n': n,
        'num_bins': num_bins,
        'chi2_stat': chi2_stat,
        'df': df,
        'critical_value': critical_value,
        'p_value': p_value,
        'reject_null': reject_null,
        'alpha': alpha
    }

def print_chi_square_results(results):
    """Affiche les résultats du test du khi-deux"""
    print(f"Taille de l'échantillon : {results['n']}")
    print(f"Nombre de classes : {results['num_bins']}")
    print(f"Degrés de liberté : {results['df']}")
    print(f"Statistique χ² : {results['chi2_stat']:.4f}")
    print(f"Valeur critique (α={results['alpha']}) : {results['critical_value']:.4f}")
    print(f"P-value : {results['p_value']:.6f}")
    print(f"Décision : {'REJETÉ' if results['reject_null'] else 'ACCEPTÉ'}")
    
    if results['reject_null']:
        print("❌ Le générateur NE PASSE PAS le test du khi-deux")
    else:
        print("✅ Le générateur PASSE le test du khi-deux")

## 4. Test spectral 2D (doublets)

Le test spectral 2D analyse les corrélations entre paires de nombres consécutifs en les visualisant dans un plan 2D.

In [ ]:
def spectral_test_2d(data, num_points_to_plot=5000):
    """
    Test spectral 2D sur des doublets
    
    Parameters:
    -----------
    data : array-like
        Séquence de nombres aléatoires
    num_points_to_plot : int
        Nombre de points à afficher sur le graphique
    
    Returns:
    --------
    dict : Résultats incluant corrélation et graphique
    """
    # Créer des doublets (x_i, x_{i+1})
    x = data[:-1]
    y = data[1:]
    
    # Calculer la corrélation
    correlation = np.corrcoef(x, y)[0, 1]
    
    # Visualisation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot des doublets
    n_plot = min(num_points_to_plot, len(x))
    ax1.scatter(x[:n_plot], y[:n_plot], alpha=0.3, s=1)
    ax1.set_xlabel('$x_i$')
    ax1.set_ylabel('$x_{i+1}$')
    ax1.set_title(f'Test spectral 2D - Doublets\n(n={len(data)}, corrélation={correlation:.6f})')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    
    # Heatmap 2D
    h = ax2.hist2d(x, y, bins=50, cmap='YlOrRd')
    ax2.set_xlabel('$x_i$')
    ax2.set_ylabel('$x_{i+1}$')
    ax2.set_title('Densité des doublets')
    plt.colorbar(h[3], ax=ax2)
    
    plt.tight_layout()
    
    return {
        'correlation': correlation,
        'n_pairs': len(x),
        'figure': fig
    }

def evaluate_2d_test(correlation, threshold=0.1):
    """
    Évalue le résultat du test spectral 2D
    Une corrélation proche de 0 est attendue pour un bon générateur
    """
    print(f"Corrélation entre doublets : {correlation:.6f}")
    
    if abs(correlation) < threshold:
        print(f"✅ Le générateur PASSE le test spectral 2D (|corrélation| < {threshold})")
        return True
    else:
        print(f"❌ Le générateur NE PASSE PAS le test spectral 2D (|corrélation| >= {threshold})")
        return False

## 5. Test spectral 3D (triplets)

Le test spectral 3D analyse les corrélations entre triplets de nombres consécutifs en les visualisant dans un espace 3D. C'est un test particulièrement révélateur pour RANDU, qui est connu pour avoir des structures en plans parallèles.

In [ ]:
def spectral_test_3d(data, num_points_to_plot=5000):
    """
    Test spectral 3D sur des triplets
    
    Parameters:
    -----------
    data : array-like
        Séquence de nombres aléatoires
    num_points_to_plot : int
        Nombre de points à afficher sur le graphique 3D
    
    Returns:
    --------
    dict : Résultats incluant les triplets et graphique
    """
    # Créer des triplets (x_i, x_{i+1}, x_{i+2})
    x = data[:-2]
    y = data[1:-1]
    z = data[2:]
    
    # Calculer les corrélations
    corr_xy = np.corrcoef(x, y)[0, 1]
    corr_xz = np.corrcoef(x, z)[0, 1]
    corr_yz = np.corrcoef(y, z)[0, 1]
    
    # Visualisation 3D
    fig = plt.figure(figsize=(16, 12))
    
    # Vue 3D principale
    ax1 = fig.add_subplot(2, 2, 1, projection='3d')
    n_plot = min(num_points_to_plot, len(x))
    ax1.scatter(x[:n_plot], y[:n_plot], z[:n_plot], 
                c=z[:n_plot], cmap='viridis', alpha=0.4, s=1)
    ax1.set_xlabel('$x_i$')
    ax1.set_ylabel('$x_{i+1}$')
    ax1.set_zlabel('$x_{i+2}$')
    ax1.set_title(f'Test spectral 3D - Triplets (n={len(data)})')
    
    # Projections 2D
    ax2 = fig.add_subplot(2, 2, 2)
    ax2.scatter(x[:n_plot], y[:n_plot], alpha=0.3, s=1)
    ax2.set_xlabel('$x_i$')
    ax2.set_ylabel('$x_{i+1}$')
    ax2.set_title(f'Projection XY (corr={corr_xy:.4f})')
    ax2.grid(True, alpha=0.3)
    
    ax3 = fig.add_subplot(2, 2, 3)
    ax3.scatter(x[:n_plot], z[:n_plot], alpha=0.3, s=1)
    ax3.set_xlabel('$x_i$')
    ax3.set_ylabel('$x_{i+2}$')
    ax3.set_title(f'Projection XZ (corr={corr_xz:.4f})')
    ax3.grid(True, alpha=0.3)
    
    ax4 = fig.add_subplot(2, 2, 4)
    ax4.scatter(y[:n_plot], z[:n_plot], alpha=0.3, s=1)
    ax4.set_xlabel('$x_{i+1}$')
    ax4.set_ylabel('$x_{i+2}$')
    ax4.set_title(f'Projection YZ (corr={corr_yz:.4f})')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    return {
        'n_triplets': len(x),
        'corr_xy': corr_xy,
        'corr_xz': corr_xz,
        'corr_yz': corr_yz,
        'figure': fig
    }

def evaluate_3d_test(results, threshold=0.1):
    """
    Évalue le résultat du test spectral 3D
    """
    print(f"Corrélations entre triplets :")
    print(f"  - (x_i, x_{{i+1}}) : {results['corr_xy']:.6f}")
    print(f"  - (x_i, x_{{i+2}}) : {results['corr_xz']:.6f}")
    print(f"  - (x_{{i+1}}, x_{{i+2}}) : {results['corr_yz']:.6f}")
    
    max_corr = max(abs(results['corr_xy']), abs(results['corr_xz']), abs(results['corr_yz']))
    
    if max_corr < threshold:
        print(f"✅ Le générateur PASSE le test spectral 3D (max|corrélation| < {threshold})")
        return True
    else:
        print(f"❌ Le générateur NE PASSE PAS le test spectral 3D (max|corrélation| >= {threshold})")
        return False

## 6. Tests sur 30 000 données

In [ ]:
print("="*70)
print("TESTS AVEC 30 000 DONNÉES")
print("="*70)

# Génération des données
data_30k = randu_generator(12345, 30000)

print(f"\n✓ Généré {len(data_30k)} nombres")
print(f"  Min: {data_30k.min():.6f}, Max: {data_30k.max():.6f}")
print(f"  Moyenne: {data_30k.mean():.6f} (attendu: 0.5)")
print(f"  Écart-type: {data_30k.std():.6f} (attendu: {1/np.sqrt(12):.6f})")

### 6.1. Test du Khi-deux (30 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST DU KHI-DEUX (30 000 données)")
print("-"*70 + "\n")

results_chi2_30k = chi_square_test(data_30k, num_bins=100, alpha=0.01)
print_chi_square_results(results_chi2_30k)

# Visualisation de la distribution
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(data_30k, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.axhline(y=1, color='r', linestyle='--', label='Uniforme théorique')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.title('Distribution des 30 000 données')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
observed, bins = np.histogram(data_30k, bins=100, range=(0, 1))
expected = len(data_30k) / 100
plt.bar(range(100), observed, alpha=0.7, label='Observé')
plt.axhline(y=expected, color='r', linestyle='--', label='Attendu')
plt.xlabel('Classe')
plt.ylabel('Fréquence')
plt.title('Fréquences observées vs attendues')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.2. Test spectral 2D (30 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST SPECTRAL 2D (30 000 données)")
print("-"*70 + "\n")

results_2d_30k = spectral_test_2d(data_30k, num_points_to_plot=5000)
evaluate_2d_test(results_2d_30k['correlation'])
plt.show()

### 6.3. Test spectral 3D (30 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST SPECTRAL 3D (30 000 données)")
print("-"*70 + "\n")

results_3d_30k = spectral_test_3d(data_30k, num_points_to_plot=5000)
evaluate_3d_test(results_3d_30k)
plt.show()

## 7. Tests sur 90 000 données

In [ ]:
print("\n\n" + "="*70)
print("TESTS AVEC 90 000 DONNÉES")
print("="*70)

# Génération des données
data_90k = randu_generator(12345, 90000)

print(f"\n✓ Généré {len(data_90k)} nombres")
print(f"  Min: {data_90k.min():.6f}, Max: {data_90k.max():.6f}")
print(f"  Moyenne: {data_90k.mean():.6f} (attendu: 0.5)")
print(f"  Écart-type: {data_90k.std():.6f} (attendu: {1/np.sqrt(12):.6f})")

### 7.1. Test du Khi-deux (90 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST DU KHI-DEUX (90 000 données)")
print("-"*70 + "\n")

results_chi2_90k = chi_square_test(data_90k, num_bins=100, alpha=0.01)
print_chi_square_results(results_chi2_90k)

# Visualisation de la distribution
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(data_90k, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.axhline(y=1, color='r', linestyle='--', label='Uniforme théorique')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.title('Distribution des 90 000 données')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
observed, bins = np.histogram(data_90k, bins=100, range=(0, 1))
expected = len(data_90k) / 100
plt.bar(range(100), observed, alpha=0.7, label='Observé')
plt.axhline(y=expected, color='r', linestyle='--', label='Attendu')
plt.xlabel('Classe')
plt.ylabel('Fréquence')
plt.title('Fréquences observées vs attendues')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 7.2. Test spectral 2D (90 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST SPECTRAL 2D (90 000 données)")
print("-"*70 + "\n")

results_2d_90k = spectral_test_2d(data_90k, num_points_to_plot=5000)
evaluate_2d_test(results_2d_90k['correlation'])
plt.show()

### 7.3. Test spectral 3D (90 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST SPECTRAL 3D (90 000 données)")
print("-"*70 + "\n")

results_3d_90k = spectral_test_3d(data_90k, num_points_to_plot=5000)
evaluate_3d_test(results_3d_90k)
plt.show()

## 8. Tests sur 300 000 données

In [ ]:
print("\n\n" + "="*70)
print("TESTS AVEC 300 000 DONNÉES")
print("="*70)

# Génération des données
data_300k = randu_generator(12345, 300000)

print(f"\n✓ Généré {len(data_300k)} nombres")
print(f"  Min: {data_300k.min():.6f}, Max: {data_300k.max():.6f}")
print(f"  Moyenne: {data_300k.mean():.6f} (attendu: 0.5)")
print(f"  Écart-type: {data_300k.std():.6f} (attendu: {1/np.sqrt(12):.6f})")

### 8.1. Test du Khi-deux (300 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST DU KHI-DEUX (300 000 données)")
print("-"*70 + "\n")

results_chi2_300k = chi_square_test(data_300k, num_bins=100, alpha=0.01)
print_chi_square_results(results_chi2_300k)

# Visualisation de la distribution
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(data_300k, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.axhline(y=1, color='r', linestyle='--', label='Uniforme théorique')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.title('Distribution des 300 000 données')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
observed, bins = np.histogram(data_300k, bins=100, range=(0, 1))
expected = len(data_300k) / 100
plt.bar(range(100), observed, alpha=0.7, label='Observé')
plt.axhline(y=expected, color='r', linestyle='--', label='Attendu')
plt.xlabel('Classe')
plt.ylabel('Fréquence')
plt.title('Fréquences observées vs attendues')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 8.2. Test spectral 2D (300 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST SPECTRAL 2D (300 000 données)")
print("-"*70 + "\n")

results_2d_300k = spectral_test_2d(data_300k, num_points_to_plot=5000)
evaluate_2d_test(results_2d_300k['correlation'])
plt.show()

### 8.3. Test spectral 3D (300 000 données)

In [ ]:
print("\n" + "-"*70)
print("TEST SPECTRAL 3D (300 000 données)")
print("-"*70 + "\n")

results_3d_300k = spectral_test_3d(data_300k, num_points_to_plot=5000)
evaluate_3d_test(results_3d_300k)
plt.show()

## 9. Synthèse des résultats

In [ ]:
import pandas as pd

print("\n\n" + "="*70)
print("SYNTHÈSE DES RÉSULTATS")
print("="*70 + "\n")

# Créer un tableau de synthèse
results_summary = pd.DataFrame({
    'Taille échantillon': ['30 000', '90 000', '300 000'],
    'Test χ² (passé)': [
        '✅' if not results_chi2_30k['reject_null'] else '❌',
        '✅' if not results_chi2_90k['reject_null'] else '❌',
        '✅' if not results_chi2_300k['reject_null'] else '❌'
    ],
    'χ² stat': [
        f"{results_chi2_30k['chi2_stat']:.2f}",
        f"{results_chi2_90k['chi2_stat']:.2f}",
        f"{results_chi2_300k['chi2_stat']:.2f}"
    ],
    'P-value χ²': [
        f"{results_chi2_30k['p_value']:.4f}",
        f"{results_chi2_90k['p_value']:.4f}",
        f"{results_chi2_300k['p_value']:.4f}"
    ],
    'Corr 2D': [
        f"{results_2d_30k['correlation']:.6f}",
        f"{results_2d_90k['correlation']:.6f}",
        f"{results_2d_300k['correlation']:.6f}"
    ],
    'Test 2D (passé)': [
        '✅' if abs(results_2d_30k['correlation']) < 0.1 else '❌',
        '✅' if abs(results_2d_90k['correlation']) < 0.1 else '❌',
        '✅' if abs(results_2d_300k['correlation']) < 0.1 else '❌'
    ],
    'Corr 3D (max)': [
        f"{max(abs(results_3d_30k['corr_xy']), abs(results_3d_30k['corr_xz']), abs(results_3d_30k['corr_yz'])):.6f}",
        f"{max(abs(results_3d_90k['corr_xy']), abs(results_3d_90k['corr_xz']), abs(results_3d_90k['corr_yz'])):.6f}",
        f"{max(abs(results_3d_300k['corr_xy']), abs(results_3d_300k['corr_xz']), abs(results_3d_300k['corr_yz'])):.6f}"
    ],
    'Test 3D (passé)': [
        '✅' if max(abs(results_3d_30k['corr_xy']), abs(results_3d_30k['corr_xz']), abs(results_3d_30k['corr_yz'])) < 0.1 else '❌',
        '✅' if max(abs(results_3d_90k['corr_xy']), abs(results_3d_90k['corr_xz']), abs(results_3d_90k['corr_yz'])) < 0.1 else '❌',
        '✅' if max(abs(results_3d_300k['corr_xy']), abs(results_3d_300k['corr_xz']), abs(results_3d_300k['corr_yz'])) < 0.1 else '❌'
    ]
})

print(results_summary.to_string(index=False))

print("\n\n" + "-"*70)
print("CONCLUSION")
print("-"*70)
print("""
Le générateur RANDU est un générateur congruentiel linéaire historiquement 
défaillant, particulièrement connu pour ses défauts dans le test spectral 3D.

Les triplets de nombres générés par RANDU se trouvent sur seulement 15 plans 
parallèles dans l'espace 3D, ce qui révèle une structure très régulière et 
non aléatoire.

Ce générateur peut passer le test du khi-deux (qui teste uniquement la 
distribution marginale), mais échoue aux tests spectraux qui révèlent les 
corrélations entre valeurs successives.

⚠️ RANDU ne doit JAMAIS être utilisé dans des applications réelles nécessitant
des nombres pseudo-aléatoires de qualité.
""")

## 10. Comparaison avec un générateur de référence (bonus)

Pour mieux comprendre les défauts de RANDU, comparons-le avec le générateur de NumPy (Mersenne Twister / PCG64).

In [ ]:
print("="*70)
print("COMPARAISON: RANDU vs NumPy (générateur de référence)")
print("="*70)

# Générer des données avec NumPy
np.random.seed(12345)
data_numpy = np.random.random(30000)

# Test spectral 3D
fig = plt.figure(figsize=(16, 6))

# RANDU
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
n = 5000
ax1.scatter(data_30k[:-2][:n], data_30k[1:-1][:n], data_30k[2:][:n], 
            c=data_30k[2:][:n], cmap='viridis', alpha=0.4, s=1)
ax1.set_xlabel('$x_i$')
ax1.set_ylabel('$x_{i+1}$')
ax1.set_zlabel('$x_{i+2}$')
ax1.set_title('RANDU - Structure en plans parallèles\n(générateur DÉFAILLANT)')

# NumPy
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.scatter(data_numpy[:-2][:n], data_numpy[1:-1][:n], data_numpy[2:][:n], 
            c=data_numpy[2:][:n], cmap='viridis', alpha=0.4, s=1)
ax2.set_xlabel('$x_i$')
ax2.set_ylabel('$x_{i+1}$')
ax2.set_zlabel('$x_{i+2}$')
ax2.set_title('NumPy - Répartition homogène\n(générateur de QUALITÉ)')

plt.tight_layout()
plt.show()

print("\n✓ Différence visible: RANDU montre des plans parallèles évidents,")
print("  tandis que NumPy produit une distribution uniforme dans l'espace 3D.")